# Fine-Tuning BERT

## This notebook serves to fine tune a BERT model. The base model is from HuggingFace.

### NLP Task: named entity recognition
### Dataset: Contract Understanding Atticus Dataset (CUAD)
### Loading pre-trained model
### Monitor training
### Deliverable: Submit the code for fine-tuning, training logs, and a short analysis of the results.
### SOURCE: https://medium.com/@vcjayan2013/handling-cuad-dataset-for-legal-entity-extraction-the-bert-way-bee0c9d703e0


In [16]:
import pandas as pd
import numpy as np
import json
import re, os, itertools
from tqdm import tqdm
import json
import nltk
#nltk.download() #https://www.nltk.org/data.html
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

Text cleaning function below

In [17]:
#define function for text cleaning
import re

def clean(text):
    text = text.replace("\n", " ")

    # Use raw strings for regex patterns
    text = re.sub(r"_", " ", text, 0)
    text = re.sub(r"-{5,}", " ", text, 0)  # match 5 or more dashes
    text = re.sub(r"\*+", "*", text, 0)
    text = re.sub(r" \.\ ", ".", text, 0)

    text = text.strip()
    return text


Defining the path. Then we walk to iterate over the files. We get the file names, then we append the file text. Then save a dataframe of this info

Looking at dataframe

In [18]:

corpus = "CUAD_v1/full_contract_txt"
file = []
contract = []

for i in os.listdir(corpus):
    path = f"{corpus}/{i}"
    if os.path.isfile(path):
        file.append(i)
        with open(path, 'r', errors='ignore') as f:
            text = f.read()
            text = clean(text)
            contract.append(text)

df_final = pd.DataFrame({'file': file, 'contract': contract})

/var/folders/s8/0lgl11h90m15yr0rhlr9l94h0000gn/T/ipykernel_77145/603832585.py:8: DeprecationWarning: 'count' is passed as positional argument
  text = re.sub(r"_", " ", text, 0)
/var/folders/s8/0lgl11h90m15yr0rhlr9l94h0000gn/T/ipykernel_77145/603832585.py:9: DeprecationWarning: 'count' is passed as positional argument
  text = re.sub(r"-{5,}", " ", text, 0)  # match 5 or more dashes
/var/folders/s8/0lgl11h90m15yr0rhlr9l94h0000gn/T/ipykernel_77145/603832585.py:10: DeprecationWarning: 'count' is passed as positional argument
  text = re.sub(r"\*+", "*", text, 0)
/var/folders/s8/0lgl11h90m15yr0rhlr9l94h0000gn/T/ipykernel_77145/603832585.py:11: DeprecationWarning: 'count' is passed as positional argument
  text = re.sub(r" \.\ ", ".", text, 0)


In [19]:
df_final.head(8)

,file,contract
0,LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGRE...,EXHIBIT 10.6 DI...
1,"WHITESMOKE,INC_11_08_2011-EX-10.26-PROMOTION A...",Exhibit 10.26 CONFIDENTIAL TREATMENT HAS BE...
2,LohaCompanyltd_20191209_F-1_EX-10.16_11917878_...,Exhibit 10.16 SUPPLY CONTRACT Contract No: Dat...
3,CENTRACKINTERNATIONALINC_10_29_1999-EX-10.3-WE...,1 ...
4,NELNETINC_04_08_2020-EX-1-JOINT FILING AGREEME...,Exhibit 1 JOINT FILING AGREEMENT The undersi...
5,ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT A...,REDACTED COPY CONFIDENTIAL TREATMENT REQUESTE...
6,"KIROMICBIOPHARMA,INC_05_11_2020-EX-10.23-CONSU...",Exhibit 10.23 Corporate Address Fannin South P...
7,"VEONEER,INC_02_21_2020-EX-10.11-JOINT VENTURE ...",Exhibit 10.11 AMENDMENT AND TERMINATION OF ...


From the medium article, the author decided to trim to 5000 characters as the agreement date will almost certainly be in that snippet.

In [20]:
df_final['contract'] = df_final['contract'].str.slice(0, 5000)

Text Annotation section

In [21]:
import os
from tqdm import tqdm

json_file = []

for index, row in tqdm(df_final.iterrows(), total=df_final.shape[0]):
    text = row['contract']
    labels = []  # doing annotation later
    json_file.append({'text': text, 'labels': labels})


output_dir = "output"
os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist
json_path = os.path.join(output_dir, "json_file.json")

# Write the JSONL (Doccano format)
with open(json_path, 'w', encoding='utf-8') as f:
    for entry in json_file:
        f.write(json.dumps(entry, ensure_ascii=False) + '\n')


100%|██████████| 510/510 [00:00<00:00, 75704.10it/s]


# The below section is now working with labels. This should potentially be in a different notebook, but for the sake of this externship I will leave everything in this one.

In [23]:
import os
import pandas as pd

data_dir = "data"
jsonl_file = os.path.join(data_dir, "doccano", "admin.jsonl")

df = pd.read_json(jsonl_file, lines=True)
df.head()

/var/folders/s8/0lgl11h90m15yr0rhlr9l94h0000gn/T/ipykernel_77145/2999507829.py:7: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(jsonl_file, lines=True)


ValueError: Expected object or value